# 可能的探究方向

## 目录
* [PINN](#pinn)
    * [1.1-原理](#1-1)
    * [1.2-实现](#1-2)
    * [1.3-参考](#1-3)
* [DeepONet](#deeponet)
    * [2.1-原理](#2-1)
    * [2.2-实现](#2-2)
    * [2.3-参考](#2-3)
* [FNO](#fno)
    * [3.1-参考](#3-1)
* [神经网络与其它方法的联系](#relation)
  

## PINN <a id='pinn'></a>
物理信息神经网络 [[2](https://www.sciencedirect.com/science/article/pii/S0021999118307125)] 通过使用自动微分将偏微分方程 (PDE) 嵌入到神经网络的损失函数中，集成来自观测和偏微分方程 (PDE) 的信息。偏微分方程可以是整数阶偏微分方程[[2](https://www.sciencedirect.com/science/article/pii/S0021999118307125)]、积分微分方程[[3](https://epubs.siam.org/doi/10.1137/19M1274067)]或随机偏微分方程[[4](https://epubs.siam.org/doi/10.1137/19M1260141)]

### 1.1-原理 <a id='1-1'></a>
下面以求解以为Burgers方程作为PINN求解正向问题的例子。首先给出Burgers方程的定义
$$
\frac{\partial u}{\partial t}+u \frac{\partial u}{\partial x}-\nu \frac{\partial^{2} u}{\partial x^{2}}=0
$$
损失函数包括初始和边界条件下 $u$ 的有监督损失和Burgers方程的无监督损失:
$$
\begin{aligned}
\mathcal{L} &=w_{data}\mathcal{L}_{\text {data}}+w_{PDE}\mathcal{L}_{\text {PDE}} \\
\end{aligned}
$$
其中
$$
\begin{aligned}
\mathcal{L}_{\text {data}} &=\frac{1}{N_{data}} \sum_{i=1}^{N_{data}}\left|u\left(x_{i}, t_{i}\right)-u_{i}\right|^{2} \\
\mathcal{L}_{\text {PDE}} &=\frac{1}{N_{PDE}} \sum_{j=1}^{N_{PDE}}\left|\frac{\partial u}{\partial t}+u \frac{\partial u}{\partial x}-\nu \frac{\partial^{2} u}{\partial x^{2}}\right|^{2}_{(x_{j}, t_{j})}
\end{aligned}
$$
这里 $\left\{(x_{i}, t_{i})\right\} $ 和 $\left\{(x_{j}, t_{j})\right\}$ 分别是在初始/边界位置和整个域中采样的两组点，$u_{i}$ 是 $u$ 在 $(x_{i}, t_{i})$ 处的值； $w_{data}$ 和 $w_{PDE}$ 是用于平衡两个损失项之间相互作用的权重。这些权重可以定义或自动调整，在提高 PINN 的可训练性方面发挥着重要作用。PINN 算法如下所示，有关 PINN 的更多详细信息和推荐的 Python 库 DeepXDE 可以在 [参考](#1-4)[3] 中找到

### 1.2-实现 <a id='1-2'></a>

代码链接：
- [PINN原作者github](https://github.com/paraklas/PINNs) 这个版本有点老了,是用tensorflow 1.x版本写的。他github上还有其它的一些方法。
- [PINN-pytorch版本](https://github.com/jayroxis/PINNs) 包含求解Burgers方程的正反问题
- [RFM|随机特征方法 Random Feature Method](https://nb.bohrium.dp.tech/detail/3413343451) 包含FDM、PINN、RFM的对比

### 1.3-参考 <a id='1-3'></a>

1. [Physics-informed machine learning](https://sci-hub.ru//downloads/2021-08-12/49/karniadakis2021.pdf#navpanes=0&view=FitH)
2. [Physics-informed neural networks: a deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations](https://www.sciencedirect.com/science/article/pii/S0021999118307125) 
3. [DeepXDE: a deep learning library for solving differential equations](https://epubs.siam.org/doi/10.1137/19M1274067)
4. [Learning in modal space: solving time-dependent stochastic PDEs using physics-informed neural networks](https://epubs.siam.org/doi/10.1137/19M1260141)

## DeepONet <a id='deeponet'></a>

DeepONet是一种通过神经网络逼近算子，从而能够求解一类PDE的方法。

### 2.1-原理 <a id='2-1'></a>

> **定理2.1.1** 通用算子逼近定理(Universal Approximation Theorem for operators)  假设$\sigma$ 是一个非多项式连续函数，$X$是一个$Banach$空间，$K_{1} \subset X, K_{2} \subset R^{d}$分别是$X$和$R^{d}$上的紧集。$V$是$C(K_{1})$上的紧集，$G$是$V\rightarrow C(K_{2})$的连续非线性算子。那么对于任意的$\epsilon>0$，存在常数$c_{i}^{k},\xi_{ij}^{k},\theta_{i}^{k},\eta_{k} \in R$，$w_{k}\in R^{d},x_{j}\in K_{1}$，$i=1,\dots,n,j=1,\dots,m,k=1,\dots,p$，使得
> $$
> \left|\left|G(u)(y)-\sum_{k=1}^{p}\sum_{i=1}^{n}c_{i}^{k} \sigma_{i}^{k}\sigma(w_{k}y+\eta_{k})\right|\right|<\epsilon
> $$
> 对于任意的$u\in V,y\in K_{2}$成立。其中$\sigma_{i}^{k}=\sigma(\theta_{i}^{k}+\sum_{j=1}^{m}\xi_{ij}^{k}u(x_{j}))$。

$$
\begin{aligned}
\text{Banach Space:}\quad X\supset \ &K_{1} & \quad& K_{2}\subset R^{d}\\
& \downarrow & &\downarrow \\
 C(K_{1}) \supset& V &\xrightarrow{G} \quad & C(K_{2})\\
& in & & in \\
& u &\xrightarrow{G} \quad & G(u)
\end{aligned}
$$

![DeepONet-1.png](img/DeepONet-1.png)

### 2.2-实现 <a id='2-2'></a>

代码链接：
- [DeepONet作者本人github](https://github.com/lululxvi/deeponet) 基于deepxde和tensorflow
- [知乎-偏微分方程求解器（FNO,DeepOnet)等解读](https://zhuanlan.zhihu.com/p/585290158) 包含DeepONet和FNO

### 2.3-参考 <a id='2-3'></a>

1. [Learning nonlinear operators via DeepONet based on the universal approximation theorem of operators](https://arxiv.org/abs/1910.03193)
2. [知乎解读DeepONet](https://zhuanlan.zhihu.com/p/514148390)
3. [关于DeepONet的一些变种](https://github.com/lululxvi/deepxde) 在说明文档中包含了DeepONet的一些变种，如POD-DeepONet、Fourier-DeepONet、physics-informed DeepONet等。郭柯言用的DeepONet应该就可以归类于physics-informed DeepONet。

## FNO <a id='fno'></a>


### 3.1-参考 <a id='3-1'></a>

1. [原论文](https://openreview.net/forum?id=c8P9NQVtmnO)
2. [知乎解读](https://zhuanlan.zhihu.com/p/520487599)

## 神经网络与其它方法的联系 <a id='relation'></a>
经典数值算法，例如龙格-库塔方法和有限元方法，一直是在计算机中研究和模拟物理系统的主要工具。有趣的是，许多现代深度学习模型可以通过观察与许多经典算法的明显对应关系来分析。特别是，在实践中取得巨大成功的几种网络结构与数值方法的联系。例如，卷积神经网络(CNN)类似于PDE的有限差分格式[1];在分析中引入对偶空间和数据空间的概念，有助于理解CNN中使用的各种卷积操作和池化的功能[2];残差神经网络(ResNet)类似于自治常微分方程的前向欧拉格式[3]。<br>
参考

1. [PDE-Net: learning PDEs from data](https://arxiv.org/abs/1710.09668)
2. [MgNet: a unified framework of multigrid and convolutional neural network](https://arxiv.org/pdf/1901.10415.pdf)
3. [Deep residual learning for image recognition](https://arxiv.org/pdf/1512.03385.pdf)